In [1]:
import pandas as pd
import numpy as np
import torch

torch.cuda.empty_cache()

def read_file(fname: str, correct_labels=True) -> pd.DataFrame:
    """Reads a filename, return df with text and labels."""

    df = pd.read_table(fname, sep="\t", header=None, names="text,labels,role".split(","))
    if correct_labels:
        offensive_ids = df.labels != "Acceptable speech"
        df.labels[offensive_ids] = 1
        df.labels[~offensive_ids] = 0

    df = df.drop(columns=["role"])
    return df


def run_hyperparams_optimization(model_name, model_type, lang):
    import torch
    torch.cuda.empty_cache()
    if lang not in {"sl", "hr", "en"}:
        raise AttributeError(f"Language {lang} is not valid")
    eval_file, train_file = f"../data/merged-{lang}.test.tsv" , f"../data/merged-{lang}.train.tsv"
    from simpletransformers.classification import ClassificationModel, ClassificationArgs
    import wandb


    model_args = ClassificationArgs()
    model_args.use_early_stopping = True
    model_args.early_stopping_delta = 0.01
    model_args.early_stopping_metric = "mcc"
    model_args.early_stopping_metric_minimize = False
    model_args.early_stopping_patience = 3
    model_args.evaluate_during_training_steps = 1000

    model_args.evaluate_during_training = True
    model_args.manual_seed = 4
    model_args.use_multiprocessing = True
    model_args.eval_batch_size = 8
    model_args.labels_list = [0, 1]
    model_args.wandb_project = "task3"
    model_args.reprocess_input_data = True
    model_args.overwrite_output_dir = True
    model_args.no_save = True

    sweep_config = {
        "method": "bayes",  # grid, random
        "metric": {"name": "mcc", "goal": "maximize"},
        "parameters": {
            "num_train_epochs":  {"max": 20, "min": 3},
            "learning_rate": {"min": 1e-6, "max": 1e-4},
            "train_batch_size": {"max": 100, "min": 5}
        },
    }

    sweep_id = wandb.sweep(sweep_config, project="task3_"+model_name.replace("/", "_")+model_type)

    train_df = read_file(train_file, correct_labels=True)
    eval_df = read_file(eval_file, correct_labels=True)

    def train():
        # Initialize a new wandb run
        wandb.init()

        # Create a TransformerModel
        model = ClassificationModel(
            model_type,
            model_name,
            use_cuda=True,
            args=model_args,
            sweep_config=wandb.config,
        )
        model.overwrite_output_dir = True
        model.no_save = True
        
        
        # Train the model
        model.train_model(train_df, eval_df=eval_df)

        # Evaluate the model
        model.eval_model(eval_df)

        # Sync wandb
        wandb.join()

    wandb.agent(sweep_id, train, count=50)


 "electra",
        "classla/bcms-bertic",

In [2]:
model_name = "classla/bcms-bertic"
model_type = "electra"
language = "hr"

run_hyperparams_optimization(model_name, model_type, language)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 6am13hot
Sweep URL: https://wandb.ai/5roop/task3_classla_bcms-berticelectra/sweeps/6am13hot


wandb: Agent Starting Run: 4g9eu69v with config:
wandb: 	learning_rate: 3.435642841771529e-05
wandb: 	num_train_epochs: 14
wandb: 	train_batch_size: 17
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 5roop (use `wandb login --relogin` to force relogin)


Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight

/home/peterr/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:941: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(


wandb: Ctrl + C detected. Stopping sweep.


In [3]:
model_name = "EMBEDDIA/sloberta"
model_type = "camembert"
language = "sl"

run_hyperparams_optimization(model_name, model_type, language)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


KeyboardInterrupt: 

In [7]:
model_name = "EMBEDDIA/sloberta"
model_type = "roberta"
language = "sl"

run_hyperparams_optimization(model_name, model_type, language)

Create sweep with ID: um04mk5v
Sweep URL: https://wandb.ai/5roop/task3_EMBEDDIA_slobertaroberta/sweeps/um04mk5v


wandb: Agent Starting Run: qwu4qq1k with config:
wandb: 	learning_rate: 1.5175992821295014e-05
wandb: 	num_train_epochs: 7
wandb: 	train_batch_size: 13
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


You are using a model of type camembert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at EMBEDDIA/sloberta were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification we

Run qwu4qq1k errored: OSError("Can't load tokenizer for 'EMBEDDIA/sloberta'. Make sure that:\n\n- 'EMBEDDIA/sloberta' is a correct model identifier listed on 'https://huggingface.co/models'\n\n- or 'EMBEDDIA/sloberta' is the correct path to a directory containing relevant tokenizer files\n\n")
wandb: ERROR Run qwu4qq1k errored: OSError("Can't load tokenizer for 'EMBEDDIA/sloberta'. Make sure that:\n\n- 'EMBEDDIA/sloberta' is a correct model identifier listed on 'https://huggingface.co/models'\n\n- or 'EMBEDDIA/sloberta' is the correct path to a directory containing relevant tokenizer files\n\n")
wandb: Agent Starting Run: 3hi4lnee with config:
wandb: 	learning_rate: 8.57222802142255e-05
wandb: 	num_train_epochs: 14
wandb: 	train_batch_size: 14
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


You are using a model of type camembert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at EMBEDDIA/sloberta were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification we

Run 3hi4lnee errored: OSError("Can't load tokenizer for 'EMBEDDIA/sloberta'. Make sure that:\n\n- 'EMBEDDIA/sloberta' is a correct model identifier listed on 'https://huggingface.co/models'\n\n- or 'EMBEDDIA/sloberta' is the correct path to a directory containing relevant tokenizer files\n\n")
wandb: ERROR Run 3hi4lnee errored: OSError("Can't load tokenizer for 'EMBEDDIA/sloberta'. Make sure that:\n\n- 'EMBEDDIA/sloberta' is a correct model identifier listed on 'https://huggingface.co/models'\n\n- or 'EMBEDDIA/sloberta' is the correct path to a directory containing relevant tokenizer files\n\n")
wandb: Agent Starting Run: g1tbhqt0 with config:
wandb: 	learning_rate: 9.063901189210694e-05
wandb: 	num_train_epochs: 3
wandb: 	train_batch_size: 100
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


You are using a model of type camembert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at EMBEDDIA/sloberta were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification we

Run g1tbhqt0 errored: OSError("Can't load tokenizer for 'EMBEDDIA/sloberta'. Make sure that:\n\n- 'EMBEDDIA/sloberta' is a correct model identifier listed on 'https://huggingface.co/models'\n\n- or 'EMBEDDIA/sloberta' is the correct path to a directory containing relevant tokenizer files\n\n")
wandb: ERROR Run g1tbhqt0 errored: OSError("Can't load tokenizer for 'EMBEDDIA/sloberta'. Make sure that:\n\n- 'EMBEDDIA/sloberta' is a correct model identifier listed on 'https://huggingface.co/models'\n\n- or 'EMBEDDIA/sloberta' is the correct path to a directory containing relevant tokenizer files\n\n")
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true


In [9]:
# Doesen't run, don't know why
model_name = "xlm-roberta-base"
model_type = "xlm-roberta"
language = "sl"

run_hyperparams_optimization(model_name, model_type, language)

Create sweep with ID: xtzzgzuv
Sweep URL: https://wandb.ai/5roop/task3_xlm-roberta-basexlm-roberta/sweeps/xtzzgzuv


wandb: Agent Starting Run: im3ycjw4 with config:
wandb: 	learning_rate: 7.525077012287136e-05
wandb: 	num_train_epochs: 5
wandb: 	train_batch_size: 7
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Run im3ycjw4 errored: KeyError('xlm-roberta')
wandb: ERROR Run im3ycjw4 errored: KeyError('xlm-roberta')
wandb: Agent Starting Run: un0yk9nj with config:
wandb: 	learning_rate: 4.909199593792542e-05
wandb: 	num_train_epochs: 20
wandb: 	train_batch_size: 25
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Run un0yk9nj errored: KeyError('xlm-roberta')
wandb: ERROR Run un0yk9nj errored: KeyError('xlm-roberta')
wandb: Agent Starting Run: a1lx75od with config:
wandb: 	learning_rate: 1.0999838114011426e-05
wandb: 	num_train_epochs: 12
wandb: 	train_batch_size: 98
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Run a1lx75od errored: KeyError('xlm-roberta')
wandb: ERROR Run a1lx75od errored: KeyError('xlm-roberta')
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true


In [3]:
# Doesen't run, don't know why
model_name = "xlm-roberta-large"
model_type = "xlm-roberta"
language = "sl"

run_hyperparams_optimization(model_name, model_type, language)

Create sweep with ID: ggk3zsph
Sweep URL: https://wandb.ai/5roop/task3_xlm-roberta-largexlm-roberta/sweeps/ggk3zsph


wandb: Agent Starting Run: 7w8r1h5i with config:
wandb: 	learning_rate: 9.901888495782402e-06
wandb: 	num_train_epochs: 19
wandb: 	train_batch_size: 71
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Run 7w8r1h5i errored: KeyError('xlm-roberta')
wandb: ERROR Run 7w8r1h5i errored: KeyError('xlm-roberta')
wandb: Agent Starting Run: 5c0aworn with config:
wandb: 	learning_rate: 3.609675069777099e-05
wandb: 	num_train_epochs: 18
wandb: 	train_batch_size: 69
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Run 5c0aworn errored: KeyError('xlm-roberta')
wandb: ERROR Run 5c0aworn errored: KeyError('xlm-roberta')
wandb: Agent Starting Run: ln9nvxyx with config:
wandb: 	learning_rate: 8.763866256561414e-05
wandb: 	num_train_epochs: 3
wandb: 	train_batch_size: 30
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Run ln9nvxyx errored: KeyError('xlm-roberta')
wandb: ERROR Run ln9nvxyx errored: KeyError('xlm-roberta')
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true


In [4]:
# Doesen't run, don't know why
model_name = "xlm-roberta"
model_type = "xlm-roberta"
language = "sl"

run_hyperparams_optimization(model_name, model_type, language)

Create sweep with ID: abego22b
Sweep URL: https://wandb.ai/5roop/task3_xlm-robertaxlm-roberta/sweeps/abego22b


wandb: Agent Starting Run: w869ljxa with config:
wandb: 	learning_rate: 1.8163040630492245e-05
wandb: 	num_train_epochs: 6
wandb: 	train_batch_size: 12
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Run w869ljxa errored: KeyError('xlm-roberta')
wandb: ERROR Run w869ljxa errored: KeyError('xlm-roberta')
wandb: Agent Starting Run: d8na1zvk with config:
wandb: 	learning_rate: 1.1651535465097527e-05
wandb: 	num_train_epochs: 4
wandb: 	train_batch_size: 99
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Run d8na1zvk errored: KeyError('xlm-roberta')
wandb: ERROR Run d8na1zvk errored: KeyError('xlm-roberta')
wandb: Agent Starting Run: leu1bixn with config:
wandb: 	learning_rate: 7.916020715391716e-05
wandb: 	num_train_epochs: 20
wandb: 	train_batch_size: 100
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Run leu1bixn errored: KeyError('xlm-roberta')
wandb: ERROR Run leu1bixn errored: KeyError('xlm-roberta')
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true


In [5]:
model_name = "roberta-base"
model_type = "roberta"
language = "en"

run_hyperparams_optimization(model_name, model_type, language)

Create sweep with ID: u2c0c11r
Sweep URL: https://wandb.ai/5roop/task3_roberta-baseroberta/sweeps/u2c0c11r


wandb: Agent Starting Run: byvzcvl3 with config:
wandb: 	learning_rate: 3.903550621877325e-05
wandb: 	num_train_epochs: 4
wandb: 	train_batch_size: 69
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_p

Training loss,0.17613
lr,0.0
global_step,544
_runtime,138
_timestamp,1630924519
_step,16
tp,677
tn,1199
fp,201
fn,225
mcc,0.61002


Training loss,▇▆▆█▇▅▇▄▁▂
lr,█▇▆▆▅▄▃▃▂▁
global_step,▁▂▂▂▃▄▄▅▅▆▆▇▇█
_runtime,▁▂▂▂▃▄▄▄▅▅▆▆▇████
_timestamp,▁▂▂▂▃▄▄▄▅▅▆▆▇████
_step,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇██
tp,▁▁█▅
tn,▁█▄▇
fp,█▁▅▂
fn,██▁▄
mcc,▁▆██


...

Training loss,0.8396
lr,3e-05
global_step,4000
_runtime,316
_timestamp,1630929753
_step,88
tp,0
tn,1400
fp,0
fn,902
mcc,0.0


Training loss,▃▄▅▁▆▂▄▆▆▆▇▅▅▇▆▄▄▇▅▂▆▅█▅▂▅▄▂▅▅▅▃▄▄▅▅▄▁▄▆
lr,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
tp,▁▁▁▁▁▁
tn,▁▁▁▁▁▁
fp,▁▁▁▁▁▁
fn,▁▁▁▁▁▁
mcc,▁▁▁▁▁▁


wandb: Agent Starting Run: y8uemgsb with config:
wandb: 	learning_rate: 8.77518070751763e-05
wandb: 	num_train_epochs: 16
wandb: 	train_batch_size: 28
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_p

wandb: Ctrl + C detected. Stopping sweep.
